## Libraries and functions

### Libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd.variable import Variable

### Functions

In [2]:
def underlined_text(text, char = '='):
    length = len(text)
    print(text)
    for i in range(length):
        print(char, end='')
    print()

## Dataset

### Creating the dataset

In [3]:
underlined_text("dataX")
dataX = torch.tensor([[i] for i in range(0,10)])
print("Shape = ", dataX.shape)
print(dataX)
print()
underlined_text("dataY")
dataY = torch.tensor([i for i in range(1,10)])
z = torch.tensor([0])
dataY = torch.cat((dataY, z), dim=0)
print(dataY.shape)
print(dataY)
print()
underlined_text("Sequence order: (Input => Output)")
for i in range(len(dataX)):
    print(dataX[i].item(), '=>', dataY[i].item())

dataX
=====
Shape =  torch.Size([10, 1])
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])

dataY
=====
torch.Size([10])
tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

Sequence order: (Input => Output)
0 => 1
1 => 2
2 => 3
3 => 4
4 => 5
5 => 6
6 => 7
7 => 8
8 => 9
9 => 0


### Reshaping dataX

Reshaping the dataX into a format [batch_size, seq_len, no_features].

In [4]:
underlined_text("Reshaped dataX")
X = dataX.view(dataX.shape[0], 1, 1) # [batch_size, seq_len, features]
print(X.shape)
print(X)

Reshaped dataX
torch.Size([10, 1, 1])
tensor([[[0]],

        [[1]],

        [[2]],

        [[3]],

        [[4]],

        [[5]],

        [[6]],

        [[7]],

        [[8]],

        [[9]]])


### Normalizing

In [5]:
underlined_text("Normalizing X")
X = X/float(X.shape[0])
print(X.shape)
print(X)

Normalizing X
torch.Size([10, 1, 1])
tensor([[[0.0000]],

        [[0.1000]],

        [[0.2000]],

        [[0.3000]],

        [[0.4000]],

        [[0.5000]],

        [[0.6000]],

        [[0.7000]],

        [[0.8000]],

        [[0.9000]]])


## Creating the model

In the default mode (i.e. batch_first = False), the input shape should have the shape [seq_len, batch_size, 
features]. If you want the input shape should be of the shape [batch_size, seq_len, features], set batch_first = True.

seq_len - Number of time steps in each input stream
<br>
features - Number of input features per time-step
<br>
batch_size - Size of the batches (sample size)
<br>
hidden_size - Number of RNN/LSTM blocks
<br>
num_layers - Number of hidden layers (default = 1)
<br>
<br>
<u>Input shapes</u>
<br>
Input's shape = [seq_len, batch_size, features] if batch_first = False
<br>
Input's shape = [batch_size, seq_len, features] if batch_first = True
<br>
Initial hidden layer's shape = [num_layers, batch_size, hidden_size]
<br>
<br>
<u>Hidden layer's shape</u>
<br>
[num_layers, batch_size, hidden_size]
<br>

<u>Output shape</u>
<br>
Hidden layer's shape = [num_layers, batch_size, hidden_size] (Since in an RNN, hidden state also returned as an output)
<br>
Output's shape = [seq_len, batch_size, hidden_size] if batch_first = False
<br>
Output's shape = [batch_size, seq_len, hidden_size] if batch_first = True

In [6]:
seq_len = 1
features = 1
batch_size = 10
hidden_size = 16
num_layers = 1

simple_model = nn.RNN(
    input_size=features,
    hidden_size=hidden_size,
    num_layers=num_layers,
    batch_first=True
)

underlined_text("Simple RNN Model")
print("Model = ", simple_model)
print()

underlined_text("RNN Input shapes")
inputs = Variable(X)
hidden_0 = Variable(torch.randn(num_layers, batch_size, hidden_size))
print("Input shape [batch_size, seq_len, features]        = ", inputs.shape)
print("Hidden shape [num_layers, batch_size, hidden_size] = ", hidden_0.shape)
print()

underlined_text("RNN Output shapes")
rnn_output, hidden_i = simple_model(inputs, hidden_0)
print("Hidden shape [num_layers, batch_size, hidden_size] = ", hidden_i.shape)
print("Output shape [batch_size, seq_len, hidden_size]]   = ", rnn_output.shape, "(batch_first = True)")
print()

underlined_text("RNN's output passes to a linear layer")
reshaped_rnn_out = rnn_output.view(10,16)
print("Reshaped RNN output - shape (Input to the linear layer) = ", reshaped_rnn_out.shape)
linear_model = nn.Linear(in_features=hidden_size, out_features=10) # 10 = No of digits
final_out = linear_model(reshaped_rnn_out)
print("Linear layer output shape = ", final_out.shape)
print("Linear layer output = ", final_out)
print()

underlined_text("Obtain the max values and its index (predicted class) of the output")
max_vals, pred_classes = torch.max(final_out, dim=1)
print("Maximum values of final_out = ", max_vals)
print("Predicted classes = ", pred_classes)
print()

underlined_text("Interpretation (after one epoch - no training)")
for i in range(pred_classes.shape[0]):
    print("Next digit of ", i, " => ", pred_classes[i].item())

Simple RNN Model
Model =  RNN(1, 16, batch_first=True)

RNN Input shapes
Input shape [batch_size, seq_len, features]        =  torch.Size([10, 1, 1])
Hidden shape [num_layers, batch_size, hidden_size] =  torch.Size([1, 10, 16])

RNN Output shapes
Hidden shape [num_layers, batch_size, hidden_size] =  torch.Size([1, 10, 16])
Output shape [batch_size, seq_len, hidden_size]]   =  torch.Size([10, 1, 16]) (batch_first = True)

RNN's output passes to a linear layer
Reshaped RNN output - shape (Input to the linear layer) =  torch.Size([10, 16])
Linear layer output shape =  torch.Size([10, 10])
Linear layer output =  tensor([[-0.0947, -0.3305,  0.4749, -0.0605,  0.1579,  0.4096,  0.5342,  0.1450,
         -0.0171,  0.1144],
        [-0.5547, -0.1087, -0.2126, -0.5227,  0.5199, -0.1150,  0.3598, -0.6230,
          0.1359, -0.4138],
        [-0.2381,  0.0216, -0.0465, -0.0487,  0.0892,  0.2450,  0.2186, -0.5899,
         -0.2719, -0.0571],
        [-0.3922, -0.0576, -0.1196, -0.3911,  0.2179, -0.